TODO
* Run final Utilization Tabular report after hours are finalized for 2020-21, deleting anything after march 31; update report to filter by dates for PY 2022

This notebook demonstrates the process for updating the Utilizaton Report (ei-utilization.herokuapp.com) and Quarter Resource Planners. Reports for current data are pulled from Deltek via Cognos weekly on Mondays (and as needed). Historicals are stored in the Utilization-Hours google sheet (may be migrated to PostreSQL) raw and as processed. Because timecodes and accounting practices change each year, processed data are less likely to break scripts whereas raw data are maintained to reprocess if necessary.

Going forward, data will be processed for each planning year separately. Data from previous planning years will be finalized after timesheets are finalized in April. The hours entries and timetable for each year will be stored in tabs in the same workbook.

In [1]:
import pygsheets
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import sys
import json

# Sources of error

There are a few potential sources of error with the translation of Deltek information to this project:

    1. Employee email addresses not specified: if an employee is onboarded and their EI email address is not included in their Deltek record, they will be unable to login to the Utilization Report.
    2. New User Defined Code 3 added: if a new User Defined Code 3 (i.e., SRV for Billable, HOL for Time Off) is added, the function `code_hours_entries` will use the new User Defined Code 3 rather than the desired reclassed code.
    3. The 4-digit project ID associated with a timecode is incorrect: this is very unlikely, but the project relies on the first four digits of its project ID to associate it with the right Level 1 project (e.g., 1001=MI2).
    4. There are duplicate codes in the Projects report (this happened with Walton-SLED). An Assertion error will be thrown if there are any duplicates in the Projects report at Level 1; send to Arica to be removed.

0. Download reports from Cognos
1. Authorize google sheets
2. Get latest report from TESS
3. Save employee work schedules to google sheets
4. Update employee usernames and passwords *Register users rather than save employee list, but still open to own chart*
5. Read hours entries
6. Join codes as classification
7. Join projects, update Indirect to classification, ?update Unbillable to non-utilization? remove comments from time off
8. Save hours entries to google sheets
9. Save projects to google sheets
10. Build timetable for each employee (%FTE per month to billable, R&D, OH, ...)
11. Concat all employees' timetables
12. Save timetables to google sheets
13. Combine all hours reports for Utilization Report
14. Combine all timetables for Utilization Report
15. Upload budgets and actuals 
16. Check for greater than 40 hours to billable, print name and week if over

# 0. Download reports from Cognos

Two reports are downloaded from Cognos in Excel format with multiple sheets:
    
    1. Hours, and employee work schedules (from TESS)
    2. Activities, funded amounts and actuals (from Projects CP)

Two reports are required because data cannot be combined from the two different packages in Cognos. The reports are downloaded to my local Downloads folder, with an appended iterator to distinguish from previous copies. The most recent version of each is selected for processing and upload.

In [2]:
# Download folder where hours report will be saved from Deltek
downloads = r'C:/Users/Erik/Downloads/'
# Name of the file, excluding any appended numeric distinguisher for repeats
tess_file = 'Time and Employee WS'
hours_entries_sheet = 'Hours Entries'
employees_sheet = 'Employee WS'

funded_actuals_file = 'Projects and Budgets'
projects_sheet = 'Activities'
funded_sheet = 'Funded'
actuals_sheet = 'Actuals'

# 1. Authorize google sheets

A Google Project has been set up to serve as the back end for the project ("deltek-server"). Data are processed and stored in Google Sheets, which make data available to the Utilization Report and Quarterly Resource Planners (also built in Sheets). Three Sheets are maintained:

    1. hours-entries: stores historical and current hours entries and timetables for the Utilization Report
    2. deltek-info: stores information tables needed for both projects, including employees, all projects, and codes (e.g., R&D, Billable)
    3. funded-actuals: stores funded and actuals for Quarterly Resource Planners and Monthly Planners
   
Due to the time required to transfer the large amount of data from Utilization-Hours as the company grows and more time is included, we are considering migrating to a PostgreSQL database for the Utilization-Hours report.

The project is located [here](https://drive.google.com/drive/folders/1DJU91YLlaEhnn2b0J7UHJ1GXLUJPiIaM?usp=sharing) under the eanderson@enviroincentives.com account.
see here for instructions on setting up a Project 
https://eanderson-ei.github.io/ei-dev/deployment/google-api/

The function `auth_gspread` returns a client for connecting to this Google Project using either locally stored credentials (store in `secrets/`) or heroku credentials. 

In [3]:
funded_actuals_sh = 'funded-actuals'
funded_wks = 'funded'
actuals_wks = 'actuals'

deltek_info_sh = 'deltek-info'
employee_work_sched_wks = 'employee-ws'
codes_wks = 'codes'
projects_wks = 'projects'

hours_entries_sh = 'hours-entries'
all_hours_wks = 'all-hours'
all_tables_wks = 'all-tables'
years = [2019, 2020, 2021]
hours_sheets = [str(year) + "-hours" for year in years]
table_sheets = [str(year) + "-table" for year in years]

current_hours_wks = hours_sheets[-1]
current_table_wks = table_sheets[-1]

In [4]:
def auth_gspread():
    """Authorize Google to access the Utilization Project"""
    # creds for local development
    try:
        client = pygsheets.authorize(
            service_file='../secrets/gs_credentials.json'  # REMOVE ../ FOR DEV
            )
    # creds for heroku deployment
    except:
        client = pygsheets.authorize(
            service_account_env_var='GOOGLE_SHEETS_CREDS_JSON'
        )
        
    return client

In [5]:
client = auth_gspread()

The function `load_report` will load in a specified sheet.

In [6]:
def load_report(client, spreadsheet, sheet_title):
    """Load data (must be in tidy format) from sheet.
    Empty rows are dropped.
    :param client: client object for accessing google
    :param sheet_title: sheet name
    :returns pandas dataframe
    """
    # load data from google sheet
    sh = client.open(spreadsheet)
    wks = sh.worksheet_by_title(sheet_title)
    data = wks.get_all_records(empty_value=None)  # get_as_df can't handle empty columns
    df = pd.DataFrame(data)
    df.dropna(axis=0, how='all', inplace=True)
    
    return df

# 2. Get latest reports from TESS

The function `get_latest_file` takes as input a folder and file base name (i.e., excluding appended iterators) and selects the most recent file starting with that base name. 

In [7]:
def get_latest_file(downloads, file_name):
    # list all files in downloads
    all_files = [f for f in os.listdir(downloads) 
                 if f.startswith(file_name)]
    # get save times
    file_versions = [os.path.getmtime(os.path.join(downloads, f)) 
                     for f in all_files]
    # access file with most recent save time
    latest_file = [f for f in all_files 
                   if os.path.getmtime(os.path.join(downloads, f)) 
                   == max(file_versions)]
    
    latest_file = os.path.join(downloads, latest_file[0])
    print(latest_file)
    
    return latest_file

Two reports are read from the `Time and Employee WS` file in as pandas dataframe:

    1. Hours entries: all entries in Deltek (person x day x timecode x hours)
    2. Employees: all current employees (person, email address, active = 'Y')

# 3. Save Employee Work Schedules to GS

The employee work schedule report is used by the Staff Work Schedules worksheet in the Quarterly Resource Planners to update employee work schedules.

In [8]:
# locate Cognos report from TESS
tess_fn = get_latest_file(downloads, tess_file)

# read employees to dataframe
employeeWS_df = pd.read_excel(tess_fn, employees_sheet)

employeeWS_df.head()

C:/Users/Erik/Downloads/Time and Employee WS (3).xlsx


,Employee ID,Last Name,First Name,E-mail Address,Default Org,Hire date,Termination date,Active Flag,Work Schedule,Effective Date
0,100001,Alexandrovich,Andrew,andrew@enviroincentives.com,1.01.90,2010-04-05,NaT,Y,STD,2010-04-05
1,100002,Christen,Ellen,NaN,NaN,2015-09-21,2019-02-01,N,NaN,NaT
2,100003,Ajroud,Brittany,bajroud@enviroincentives.com,1.01.01,2016-10-18,NaT,Y,STD,2016-10-18
3,100003,Ajroud,Brittany,bajroud@enviroincentives.com,1.01.01.05,2016-10-18,NaT,Y,STD,2020-12-01
4,100004,Chandrasekaran,Chetna,cchandrasekaran@enviroincentives.com,1.01.90.03.01,2017-07-17,NaT,Y,STD,2017-07-17


In [9]:
# filter out Employee IDs < 100000
filt = employeeWS_df['Employee ID'] > 100000
employeeWS_df = employeeWS_df.loc[filt]

In [10]:
# convert Work Schedule column to percent
def process_work_schedule(row):
    if pd.isnull(row['Work Schedule']):
        return np.nan
    try:
        return int(row['Work Schedule']) / 100
    except ValueError:
        return 1.0

employeeWS_df['Work Schedule'] = employeeWS_df.apply(process_work_schedule, axis=1)

In [11]:
employeeWS_df[employeeWS_df['Work Schedule']<1.0]

,Employee ID,Last Name,First Name,E-mail Address,Default Org,Hire date,Termination date,Active Flag,Work Schedule,Effective Date
5,100005,Cook,Caroline,ccook@enviroincentives.com,1.01.01,2014-07-07,NaT,Y,0.80,2014-07-07
6,100005,Cook,Caroline,ccook@enviroincentives.com,1.01.01.02,2014-07-07,NaT,Y,0.80,2020-12-01
7,100007,Dubois,Natalie,ndubois@enviroincentives.com,1.01.01.02,2015-12-16,NaT,Y,0.75,2015-12-16
8,100008,Flower,Kathleen,kflower@enviroincentives.com,1.01.01,2017-09-05,NaT,Y,0.80,2017-09-05
9,100008,Flower,Kathleen,kflower@enviroincentives.com,1.01.01.04,2017-09-05,NaT,Y,0.80,2020-12-01
10,100009,Gambrill,Amy,agambrill@enviroincentives.com,1.01.01,2015-10-01,NaT,Y,0.80,2015-10-01
13,100010,Gibert,Tiffany,tgibert@enviroincentives.com,1.01.01.05,2018-09-25,NaT,Y,0.80,2020-10-01
14,100010,Gibert,Tiffany,tgibert@enviroincentives.com,1.01.01.05,2018-09-25,NaT,Y,0.80,2020-12-01
25,100018,Present,Teresa,tpresent@enviroincentives.com,1.01.01,2014-02-10,2020-09-30,N,0.75,2014-02-10
49,100043,Exline,Kelly,kexline@enviroincentives.com,1.01.90,2019-04-15,2021-01-07,N,0.75,2019-04-15


In [12]:
# create user name column
def first_last_to_username(df):
    # create user name column
    df['First Name'] = df['First Name'].str.strip()  # remove whitespace before Replicon first names
    df['Last Name'] = df['Last Name'].str.strip()
    df['User Name'] = df[['Last Name', 'First Name']].apply(
        lambda x: ', '.join(x), axis=1)
    
    df = df.drop(['First Name', 'Last Name'], axis=1)
    
    return df


employeeWS_df = first_last_to_username(employeeWS_df)

In [13]:
employeeWS_df

,Employee ID,E-mail Address,Default Org,Hire date,Termination date,Active Flag,Work Schedule,Effective Date,User Name
0,100001,andrew@enviroincentives.com,1.01.90,2010-04-05,NaT,Y,1.0,2010-04-05,"Alexandrovich, Andrew"
1,100002,NaN,NaN,2015-09-21,2019-02-01,N,NaN,NaT,"Christen, Ellen"
2,100003,bajroud@enviroincentives.com,1.01.01,2016-10-18,NaT,Y,1.0,2016-10-18,"Ajroud, Brittany"
3,100003,bajroud@enviroincentives.com,1.01.01.05,2016-10-18,NaT,Y,1.0,2020-12-01,"Ajroud, Brittany"
4,100004,cchandrasekaran@enviroincentives.com,1.01.90.03.01,2017-07-17,NaT,Y,1.0,2017-07-17,"Chandrasekaran, Chetna"
...,...,...,...,...,...,...,...,...,...
121,100114,khoran@enviroincentives.com,1.01.01.05,2021-02-16,NaT,Y,1.0,2021-02-16,"Horan, Katherine"
122,100115,efiorini@enviroincentives.com,1.01.01.05,2021-02-22,NaT,Y,1.0,2021-02-22,"Fiorini, Erin"
123,100116,lspanner@enviroincentives.com,1.01.01.03,2021-02-28,NaT,Y,1.0,2021-02-28,"Spanner, Lindsey"
124,100117,bables@enviroincentives.com,1.01.90.01.01,2021-03-30,NaT,Y,1.0,2021-03-30,"Ables, Brenda"


In [14]:
# add organization
employeeWS_df['Organization'] = 'Environmental Incentives'

In [15]:
employeeWS_df

,Employee ID,E-mail Address,Default Org,Hire date,Termination date,Active Flag,Work Schedule,Effective Date,User Name,Organization
0,100001,andrew@enviroincentives.com,1.01.90,2010-04-05,NaT,Y,1.0,2010-04-05,"Alexandrovich, Andrew",Environmental Incentives
1,100002,NaN,NaN,2015-09-21,2019-02-01,N,NaN,NaT,"Christen, Ellen",Environmental Incentives
2,100003,bajroud@enviroincentives.com,1.01.01,2016-10-18,NaT,Y,1.0,2016-10-18,"Ajroud, Brittany",Environmental Incentives
3,100003,bajroud@enviroincentives.com,1.01.01.05,2016-10-18,NaT,Y,1.0,2020-12-01,"Ajroud, Brittany",Environmental Incentives
4,100004,cchandrasekaran@enviroincentives.com,1.01.90.03.01,2017-07-17,NaT,Y,1.0,2017-07-17,"Chandrasekaran, Chetna",Environmental Incentives
...,...,...,...,...,...,...,...,...,...,...
121,100114,khoran@enviroincentives.com,1.01.01.05,2021-02-16,NaT,Y,1.0,2021-02-16,"Horan, Katherine",Environmental Incentives
122,100115,efiorini@enviroincentives.com,1.01.01.05,2021-02-22,NaT,Y,1.0,2021-02-22,"Fiorini, Erin",Environmental Incentives
123,100116,lspanner@enviroincentives.com,1.01.01.03,2021-02-28,NaT,Y,1.0,2021-02-28,"Spanner, Lindsey",Environmental Incentives
124,100117,bables@enviroincentives.com,1.01.90.01.01,2021-03-30,NaT,Y,1.0,2021-03-30,"Ables, Brenda",Environmental Incentives


In [57]:
# save employee ws to deltek-info
def save_to_gs(df, client, worksheet, sheet_name):
    sh = client.open(worksheet)
    wks = sh.worksheet_by_title(sheet_name)
    wks.set_dataframe(df, 'A1', fit = True)
    print (f'{sheet_name} uploaded to {worksheet}')

    
save_to_gs(employeeWS_df, client, deltek_info_sh, employee_work_sched_wks)

ServerNotFoundError: Unable to find the server at sheets.googleapis.com

# 4. Update employee usernames and passwords

The heroku app uses each employee's email from Deltek as a username and the standard password incentives. In the future, a true login system will be implemented for each employee to set their own password. However, the username should be restricted to an existing environmental incentives account.

In [229]:
# drop duplicate employee entries
e_df = employeeWS_df[[
    'Employee ID', 'User Name', 'E-mail Address', 'Active Flag'
    ]].copy()

e_df = e_df.drop_duplicates()

# drop inactive employees
filt = e_df['Active Flag'] == 'Y'
e_df = e_df.loc[filt, :].copy()

In [230]:
e_df.head()

,Employee ID,E-mail Address,Active Flag,User Name
0,100001,andrew@enviroincentives.com,Y,"Alexandrovich, Andrew"
2,100003,bajroud@enviroincentives.com,Y,"Ajroud, Brittany"
4,100004,cchandrasekaran@enviroincentives.com,Y,"Chandrasekaran, Chetna"
5,100005,ccook@enviroincentives.com,Y,"Cook, Caroline"
7,100007,ndubois@enviroincentives.com,Y,"Dubois, Natalie"


In [ ]:
def update_logins(e_df):    
    # convert to dictionary
    usernames = e_df.set_index('E-mail Address')
    emp_dict = usernames['User Name'].to_dict()
    
    # save as json to allow lookup of report based on email address
    with open('components/usernames.json', 'w') as f:
        json.dump(emp_dict, f, indent=4)
        
    # associate all emails with password 'incentives'
    #TODO: allow for custom passwords
    usernames['password'] = 'incentives'
    pass_dict = usernames['password'].to_dict()
    
    # save as json to secrets
    with open('secrets/passwords.json', 'r+') as f:
        existing_emp = json.load(f)
        for key in pass_dict.keys():
            if key and not key in existing_emp:
                print(f'UPDATE ENV VAR: {key} IS NEW!')
        f.seek(0)
        json.dump(pass_dict, f, indent=4)
        f.truncate()
    
    print('login information updated')


# update login information (replace with individual login system)
update_logins(employees_df)

# 5. Read hours entries

In [16]:
hours_entries = pd.read_excel(tess_fn, hours_entries_sheet)

# Deltek adds rows for long comments and merges cells (why are they like this?)
# remove null rows (i.e., the added row)
hours_entries = hours_entries.dropna(how='all')

In [17]:
hours_entries.head()

,Employee ID,Last Name,First Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Comments
0,100003.0,Ajroud,Brittany,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management
1,100003.0,Ajroud,Brittany,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report
2,100003.0,Ajroud,Brittany,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition
3,100003.0,Ajroud,Brittany,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training
4,100003.0,Ajroud,Brittany,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training


In [18]:
# create month and year convenience columns
hours_entries['Entry Month'] = pd.DatetimeIndex(hours_entries['Hours Date']).strftime('%b')
hours_entries['Entry Year'] = pd.DatetimeIndex(hours_entries['Hours Date']).strftime('%Y')

# create user name column
hours_entries = first_last_to_username(hours_entries)

# rename columns
hours_entries = hours_entries.rename(
                    {'Project ID': 'Task ID', 'Project Name': 'Task Name'}, 
                axis=1)

In [19]:
hours_entries.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name
0,100003.0,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management,Apr,2021,"Ajroud, Brittany"
1,100003.0,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report,Apr,2021,"Ajroud, Brittany"
2,100003.0,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition,Apr,2021,"Ajroud, Brittany"
3,100003.0,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training,Apr,2021,"Ajroud, Brittany"
4,100003.0,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training,Apr,2021,"Ajroud, Brittany"


# 6. Code hours entries

Deltek uses more specific codes (specified in column 'User Defined Code 3') than are desired. We instead use 'Billable', 'G&A', 'Time Off', 'Marketing & NBD', and 'R&D'. The codes specified in User Defined Code 3 are reclassed to these codes. Note that any new codes added to the User Defined Code 3 field in Deltek will need to be added to the 'codes' sheet in the deltek-info sheet. (If a code is not added, the deltek code will be used instead).

In [20]:
def code_hours_entries(hours_entries):
    ...
# read in codes 
codes_df = load_report(client, deltek_info_sh, codes_wks)

# convert to dictionary
codes_df = codes_df.set_index('User Defined Code 3')
codes_dict = codes_df['Code'].to_dict()

# replace User defined codes with Coes
hours_entries['Classification'] = hours_entries['User Defined Code 3'].replace(codes_dict)

#     return hours_entries

In [21]:
# hours_entries = code_hours_entries(hours_entries)

hours_entries.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name,Classification
0,100003.0,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management,Apr,2021,"Ajroud, Brittany",Billable
1,100003.0,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report,Apr,2021,"Ajroud, Brittany",Billable
2,100003.0,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition,Apr,2021,"Ajroud, Brittany",Billable
3,100003.0,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training,Apr,2021,"Ajroud, Brittany",Overhead
4,100003.0,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training,Apr,2021,"Ajroud, Brittany",Billable


# 7. Read projects table

Deltek structures projects with up to four levels of hierarchy. Level 1 is the project (e.g., MI2, NV CCS). Timecodes are in levels 2-4, however each project will bill at one and only one level. To get the project name for each timecode, simply select the first four digits of the timecode and look up that value in the projects report.

First we process the projects file and save to google sheets (so that it can also be available for quarterly resource reports), and then we look up the project name and add it to the hours entries table. Then we break up Indirect codes, which are all lumped together at level 1, using the Code added in step 6. Finally, unbillable towards a project is reclassified to Unbillable so as not to be included in utilization calculations.

In [61]:
# locate Cognos report from TESS
projects_fn = get_latest_file(downloads, funded_actuals_file)

# read employees to dataframe
projects_df = pd.read_excel(projects_fn, projects_sheet)

# remove whitespace
projects_df['Project Name'] = projects_df['Project Name'].str.strip()
projects_df['Organization Name'] = projects_df['Organization Name'].str.strip()

projects_df.head()

C:/Users/Erik/Downloads/Projects and Budgets.xlsx


,Project ID,Project Name,Organization ID,Organization Name,Active (Y/N),Level Number
0,1001,USAID Measuring Impact II,1.01.01.02,Global Adaptive Managemen,Y,1
1,1001.AFR,BI-AFR,1.01.01.04,Africa,Y,2
2,1001.AFR.001,BI-AFR,1.01.01.04,Africa,Y,3
3,1001.AFR.001.01,16.0.AFR_BuyIn_Mgmt,1.01.01.04,Africa,Y,4
4,1001.AFR.001.02,16.0.AFR_Zambia FS,1.01.01.04,Africa,Y,4


# 8. Save projects to google sheets

In [62]:
save_to_gs(projects_df, client, deltek_info_sh, projects_wks)

projects uploaded to deltek-info


# 9. Join projects to hours entries

In [242]:
def join_projects(hours_entries, projects_df):
    ...
# filter projects to Level 1 only
filt = projects_df['Level Number'] == 1
projects_df = projects_df.loc[filt, ['Project ID', 'Project Name']].copy()

# check for uniqueness of project id
assert len(projects_df) == len(projects_df['Project ID'].unique()), "Check for duplicate project IDs at Level 1"

# convert to dictionary with keys as strings
projects_df = projects_df.set_index('Project ID')
project_dict = projects_df['Project Name'].to_dict()
project_dict = {str(key): str(value) for key, value in project_dict.items()}

hours_entries['Project'] = hours_entries['Task ID'].str[:4].replace(project_dict)

# update 'Indirect' Projects to Classification
filt = hours_entries['Project'] == 'Indirect'
hours_entries.loc[filt, 'Project'] = hours_entries.loc[filt, 'Classification']

# update 'Unbillable'
filt = hours_entries['Task Name'].str.contains('Unbillable', na=False)
hours_entries.loc[filt, 'Classification'] = 'Unbillable'
    
#     return hours_entries

In [243]:
hours_entries.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name,Classification,Project
0,100003.0,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management,Apr,2021,"Ajroud, Brittany",Billable,Sustainable WASH Systems
1,100003.0,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report,Apr,2021,"Ajroud, Brittany",Billable,Sustainable WASH Systems
2,100003.0,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition,Apr,2021,"Ajroud, Brittany",Billable,Sustainable WASH Systems
3,100003.0,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training,Apr,2021,"Ajroud, Brittany",Overhead,Overhead
4,100003.0,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training,Apr,2021,"Ajroud, Brittany",Billable,Sustainable WASH Systems


# 9. Save hours entries to google sheets

In [245]:
save_to_gs(hours_entries, client, hours_entries_sh, current_hours_wks)

2021-hours uploaded to hours-entries


# 10. Build timetable for each employee

The timetable is the processed report that specifies the total FTE to each code by employee x month. The timetables are used to populate the Utilization Chart on the homepage of the Utilization report.

In [246]:
name = 'Anderson, Erik'  # use 'for loop' below for implementation

First we filter the hours_entries table for the name specified and drop any entries after the current date (e.g., when holidays are logged for staff ahead of time).

In [247]:
# filter hours_entries
def get_idv_hours_entries(hours_entries, name):
    ...


# drop hours entered after today (common with vacation, time off)
filt = (
            (hours_entries['User Name']==name) &
            (hours_entries['Hours Date'] <= dt.combine(
                dt.today().date(), dt.min.time()
                )
             ))
idf = hours_entries.loc[filt, :]

# return idf

In [248]:
idf.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name,Classification,Project
130,100022.0,1010.000.006,Walton-SLED Phase III (H),SRV,2021-04-01,0.50,check in with client,Apr,2021,"Anderson, Erik",Billable,Walton-SLED
131,100022.0,INDR.003.002.02,General OH Habitat,OVH,2021-04-01,1.00,conflict of interest training,Apr,2021,"Anderson, Erik",Overhead,Overhead
132,100022.0,INDR.027,K&DM Systems,G&A,2021-04-01,7.50,mi2 annual planner finance sections,Apr,2021,"Anderson, Erik",G&A,G&A
133,100022.0,INDR.003.002.02,General OH Habitat,OVH,2021-04-02,2.50,domestic lookback\nhubspot training,Apr,2021,"Anderson, Erik",Overhead,Overhead
135,100022.0,INDR.027,K&DM Systems,G&A,2021-04-02,6.78,mi2 annual planner finance sections,Apr,2021,"Anderson, Erik",G&A,G&A


Next we group by the year and month of the hours entry and the hours classification (e.g., billable, R&D) and sum the Entered Hours. 

In [249]:
# Calculate monthly total hours
idf_sum = (
    idf.groupby(['Entry Month', 'Entry Year', 'Classification'])['Entered Hours']
    .sum()
    .reset_index()
    .set_index(['Entry Year', 'Entry Month'])
)

In [250]:
idf_sum.head()

Classification  Entered Hours
Entry Year Entry Month                                
2021       Apr                 Billable           9.92
           Apr                      G&A          44.24
           Apr          Marketing & NBD           1.00
           Apr                 Overhead           7.40
           Apr                 Time Off           3.00

Then we pivot from tall to wide format with classification as columns and entered hours as values. We add a total column and a few helper columns for filtering by strategy year and semester.

In [251]:
def multiindex_pivot(df, columns=None, values=None):
    """https://github.com/pandas-dev/pandas/issues/23955"""
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df


# pivot table
idf_pivot = multiindex_pivot(idf_sum,
                       columns='Classification', 
                       values='Entered Hours')

# fill na as 0
idf_pivot.fillna(0, inplace=True)

# calculate total hours
idf_pivot['Total'] = idf_pivot.sum(axis=1, skipna=True)

# sort by month
idf_pivot.reset_index(inplace=True)
idf_pivot['DT'] = pd.to_datetime(idf_pivot['Entry Year'].astype(str)
                           + idf_pivot['Entry Month'], 
                           format='%Y%b')
idf_pivot.sort_values(by=['DT'], inplace=True)


# add strategic year helper column
def strategy_year(row):
    if row['DT'].month < pd.to_datetime('Apr', format='%b').month:
        return str(row['DT'].year - 1) + "-" + str(row['DT'].year)
    else:
        return str(row['DT'].year) + "-" + str(row['DT'].year + 1)


idf_pivot['Strategy Year'] = idf_pivot.apply(strategy_year, axis=1)


# add semester helper column
def semester(row):
    if (row['DT'].month < pd.to_datetime('Nov', format='%b').month and
    row['DT'].month > pd.to_datetime('Mar', format='%b').month):
        return 'Sem 1'
    else: 
        return 'Sem 2'


idf_pivot['Semester'] = 'None'

for strategy_year in idf_pivot['Strategy Year'].unique():
    filt = idf_pivot['Strategy Year'] == strategy_year
    idf_pivot.loc[filt, 'Semester'] = idf_pivot.loc[filt, :].apply(semester, axis=1)

# set index
idf_pivot.set_index(['Entry Year', 'Entry Month'], inplace=True)

In [252]:
idf_pivot.head()

,Classification,Billable,G&A,Marketing & NBD,Overhead,Time Off,Total,DT,Strategy Year,Semester
Entry Year,Entry Month,,,,,,,,,
2021,Apr,9.92,44.24,1.0,7.4,3.0,65.56,2021-04-01,2021-2022,Sem 1


We need to know the first and last day worked to update MEH for employees that start in the middle of the period. We use the first hour entry (as opposed to, say, the hire date of the employee) because sometimes there is a discrepancy between the hire date and the actual first day of work. 

In [253]:
# get first, last
def get_first_last(idf):
    ...

# get first day worked and last day worked
first_day_worked = idf['Hours Date'].min()
last_day_worked = idf['Hours Date'].max()
first_last = (first_day_worked, last_day_worked)

# return first_last

In [254]:
first_last

(Timestamp('2021-04-01 00:00:00'), Timestamp('2021-04-09 00:00:00'))

Now we add MEH based on the year and month of the rows, updating for new employees that start in the middle of a specific month.

In [255]:
# add meh
def add_meh(idf, first_last):
    ...

idf_pivot['MEH'] = idf_pivot['DT'].apply(
    lambda x: 8 * len(pd.bdate_range(x, x + pd.offsets.MonthBegin(1)))
)

# correct for employees who start in middle of the period
first_month_worked = first_last[0].strftime('%b')
first_year_worked = first_last[0].strftime('%Y')

first_month_MEH = 8 * len(pd.bdate_range(first_last[0], first_last[0] + pd.offsets.MonthBegin(1)))

idf_pivot.at[(first_year_worked, first_month_worked), 'MEH'] = first_month_MEH

# return idf_pivot

In [256]:
idf_pivot.head()

,Classification,Billable,G&A,Marketing & NBD,Overhead,Time Off,Total,DT,Strategy Year,Semester,MEH
Entry Year,Entry Month,,,,,,,,,,
2021,Apr,9.92,44.24,1.0,7.4,3.0,65.56,2021-04-01,2021-2022,Sem 1,176


'MEH hours to date' is used to calculate Utilization and FTE to date. 

The main case is a currently employed employee whose final entry was the preceding month or the current month.

The edge case is a current employee who started in the same month as last entry. (Because we exclude terminated  employees from the Utilization Report, we do not consider terminated employees).

An employee whose first entry of the month is not the first business day of the month may either have started in that month or neglected to entery hours for the first business day. Since, we cannot distinguish these two cases, we need a different way to distinguish between new employees and forgetful employees. We can use the 'MEH' column to identify when the employee is new vs forgetful. We first calculate the fte remaining for the month of the last entry and then calculate the difference between the MEH and the fte remaining, giving us the meh hours to date even for employees that start in the middle of the month. 

In [257]:
def calc_utilization(idf_pivot, first_last):
    ...
# Calculate meh_hours_to_date
fte_remaining = 8 * len(pd.bdate_range(
    first_last[1] + pd.offsets.Day(1), first_last[1] + pd.offsets.MonthBegin(1)))
last_month_worked = first_last[1].strftime('%b')
last_year_worked = first_last[1].strftime('%Y')
meh_hours = idf_pivot.loc[(last_year_worked, last_month_worked), 'MEH']
meh_hours_to_date = meh_hours - fte_remaining

Finally we calculate the utilization and FTE as percentages. For the last month worked (usually the current month but if an employee was terminated the last month worked would be their month of termination), we calculate utilization and FTE to date. The Utilization Report chart shows actuals as an 'x' and to date values as the real data for the last month (with an 'o').

In [258]:
# Calculate predicted billable hours for the current month
if 'Billable' in idf_pivot.columns:  
    current_billable = idf_pivot.loc[(last_year_worked, last_month_worked), 
                               'Billable']      
    predicted_hours = (current_billable / meh_hours_to_date) * meh_hours
# for employees with no billable time, billable and predicted hours are 0
else:  
    idf_pivot['Billable'] = predicted_hours = 0
    

# Calculate actual utilization for all months
idf_pivot['Utilization'] = idf_pivot['Billable'] / idf_pivot['MEH']

# Calculate utilization to date for this month
idf_pivot['Util to Date'] = idf_pivot['Utilization']
util_to_date = predicted_hours/meh_hours
idf_pivot.at[(last_year_worked, last_month_worked), 'Util to Date'] = (
    util_to_date
    )

# Calculate actual FTE for all months
idf_pivot['FTE'] = idf_pivot['Total'] / idf_pivot['MEH']

# Calculate FTE to date for this month
current_total = idf_pivot.loc[(last_year_worked, last_month_worked), 
                        'Total']
predicted_total = (current_total / meh_hours_to_date) * meh_hours
idf_pivot['FTE to Date'] = idf_pivot['FTE']
fte_to_date = predicted_total / meh_hours
idf_pivot.at[(last_year_worked, last_month_worked), 'FTE to Date'] = (
    fte_to_date
    )

In [259]:
idf_pivot.head()

,Classification,Billable,G&A,Marketing & NBD,Overhead,Time Off,Total,DT,Strategy Year,Semester,MEH,Utilization,Util to Date,FTE,FTE to Date
Entry Year,Entry Month,,,,,,,,,,,,,,
2021,Apr,9.92,44.24,1.0,7.4,3.0,65.56,2021-04-01,2021-2022,Sem 1,176,0.056364,0.177143,0.3725,1.170714


In [260]:
# associate name with record
idf_pivot['User Name'] = name
idf_pivot = idf_pivot.reset_index()

In [261]:
idf_pivot.head()

Classification,Entry Year,Entry Month,Billable,G&A,Marketing & NBD,Overhead,Time Off,Total,DT,Strategy Year,Semester,MEH,Utilization,Util to Date,FTE,FTE to Date,User Name
0,2021,Apr,9.92,44.24,1.0,7.4,3.0,65.56,2021-04-01,2021-2022,Sem 1,176,0.056364,0.177143,0.3725,1.170714,"Anderson, Erik"


In [262]:
timetable_list = []

for name in hours_entries['User Name'].unique():
    ...


timetable_list.append(idf_pivot)

# 11. Concat all timetables

In [263]:
# concat
timetables = pd.concat(timetable_list)

In [264]:
timetables

Classification,Entry Year,Entry Month,Billable,G&A,Marketing & NBD,Overhead,Time Off,Total,DT,Strategy Year,Semester,MEH,Utilization,Util to Date,FTE,FTE to Date,User Name
0,2021,Apr,9.92,44.24,1.0,7.4,3.0,65.56,2021-04-01,2021-2022,Sem 1,176,0.056364,0.177143,0.3725,1.170714,"Anderson, Erik"


# 12. Upload timetables to Google Sheets

In [265]:
save_to_gs(timetables, client, hours_entries_sh, current_table_wks)

2021-table uploaded to hours-entries


# 13. Combine all hours entries for Utilization Report

In [2]:
def load_report(client, spreadsheet, sheet_title):
    """Load data (must be in tidy format) from sheet.
    Empty rows are dropped.
    :param client: client object for accessing google
    :param sheet_title: sheet name
    :returns pandas dataframe
    """
    # load data from google sheet
    sh = client.open(spreadsheet)
    wks = sh.worksheet_by_title(sheet_title)
    data = wks.get_all_records(empty_value=None)  # get_as_df can't handle empty columns
    df = pd.DataFrame(data)
    df.dropna(axis=0, how='all', inplace=True)
    
    return df

In [ ]:
hours_list = []
for wks in hours_sheets:
    df = load_report(client, hours_entries_sh, wks)
    hours_list.append(df)
hours_df = pd.concat(hours_list)
save_to_gs(hours_df, hours_entries_sh, all_hours_wks)

# 14. Combine all timetables for Utilization Report

In [ ]:
tables_list = []
for wks in tables_sheets:
    df = load_report(client, hours_entries_sh, wks)
    tables_list.append(df)
tables_df = pd.concat(tables_list)
save_to_gs(tables_df, hours_entries_sh, all_tables_wks)

# 15. Upload budets and actuals

# 16. Check for greater than 40 hours in a week

In [23]:
hours_entries_week = hours_entries.copy()

In [25]:
hours_entries_week.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name,Classification
0,100003.0,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management,Apr,2021,"Ajroud, Brittany",Billable
1,100003.0,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report,Apr,2021,"Ajroud, Brittany",Billable
2,100003.0,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition,Apr,2021,"Ajroud, Brittany",Billable
3,100003.0,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training,Apr,2021,"Ajroud, Brittany",Overhead
4,100003.0,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training,Apr,2021,"Ajroud, Brittany",Billable


In [26]:
hours_entries_week['week'] = hours_entries_week['Hours Date'].dt.isocalendar().week

In [27]:
hours_entries_week.head()

,Employee ID,Task ID,Task Name,User Defined Code 3,Hours Date,Entered Hours,Comments,Entry Month,Entry Year,User Name,Classification,week
0,100003.0,1005.004.001,SWS Task 0 Management,SRV,2021-04-01,1.0,Email/task management,Apr,2021,"Ajroud, Brittany",Billable,13
1,100003.0,1005.004.005,SWS Task 3b Reporting,SRV,2021-04-01,4.0,Semiannual progress report,Apr,2021,"Ajroud, Brittany",Billable,13
2,100003.0,1005.004.006,SWS Task 4 CKM,SRV,2021-04-01,2.0,Messaging workshop planning; CLA case competition,Apr,2021,"Ajroud, Brittany",Billable,13
3,100003.0,INDR.003.001.05,General OH Comms & KM,OVH,2021-04-01,1.0,Conflict of Interest Training,Apr,2021,"Ajroud, Brittany",Overhead,13
4,100003.0,1005.004.002,SWS Task 1 Learning Event,SRV,2021-04-02,0.5,SNA training,Apr,2021,"Ajroud, Brittany",Billable,13


In [33]:
filt = hours_entries_week['Classification'] == 'Billable'
weekly_hours = hours_entries_week.loc[filt].groupby(['Employee ID','week'])['Entered Hours'].sum()
weekly_hours.head()

Employee ID  week
100003.0     13      14.50
             14      16.50
100007.0     13       1.25
             14      13.00
100009.0     13       5.50
Name: Entered Hours, dtype: float64

In [34]:
filt = weekly_hours>40
weekly_hours.loc[filt]

Employee ID  week
100090.0     14      41.0
100097.0     14      72.0
Name: Entered Hours, dtype: float64